# Accessing an AutoAI Model
In this notebook, we use the Watson Machine Learning (WML) API to find the available models available, and find the availablr deployments.

We then score some records using a Churn model.

Finally, we show how a deployment could be removed.

See:<br/>https://wml-api-pyclient-dev-v4.mybluemix.net/<br/>
https://watson-ml-v4-api.mybluemix.net/<br/>
https://wml-api-pyclient.mybluemix.net/

In [1]:
# Install the Watson Machine Learning API client
!pip install watson-machine-learning-client-v4

     |████████████████████████████████| 1.2MB 7.5MB/s eta 0:00:01
     |████████████████████████████████| 61kB 21.0MB/s eta 0:00:01
     |████████████████████████████████| 768kB 36.8MB/s eta 0:00:01
     |████████████████████████████████| 225kB 38.5MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/37/9c/c4/a2c610ccb877d37c2cb87a5bfe55845fecffd6bb01bcd5e9d5
  Stored in directory: /home/dsxuser/.cache/pip/wheels/75/93/e6/23071b2c037147a0993d34b64a03e51abca84435fc9cd6a278
  Stored in directory: /home/dsxuser/.cache/pip/wheels/23/d9/d7/43fd95b014eed89466154d8373bf4cffbb3d972de7841e213c
Successfully built ibm-cos-sdk ibm-cos-sdk-core ibm-cos-sdk-s3transfer
  Found existing installation: ibm-cos-sdk-core 2.4.3
    Uninstalling ibm-cos-sdk-core-2.4.3:
      Successfully uninstalled ibm-cos-sdk-core-2.4.3
  Found existing installation: ibm-cos-sdk-s3transfer 2.4.3
    Uninstalling ibm-cos-sdk-s3transfer-2.4.3:
      Successfully uninstalled ibm-cos-sdk-s3transfer-2.4.3
  

In [2]:
wml_credentials = {
  "apikey": "QJvpz0gQ9O_P5hktrWs97G9X4IfeSjzrxtdiC708FPPh",
  "iam_apikey_description": "Auto-generated for key 45288ebd-b9c0-4f20-9c40-5f9c335a8315",
  "iam_apikey_name": "wdp-writer",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/ec53367407ea4efb8810cf8f6fb0322d::serviceid:ServiceId-8329877a-12a2-4706-a6d1-1ef5e653fbf5",
  "instance_id": "6823c5e2-42a0-48f4-a183-7340cf086c7d",
  "url": "https://us-south.ml.cloud.ibm.com"
}

In [3]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

client = WatsonMachineLearningAPIClient(wml_credentials)

In [4]:
# List models already in the repository
client.repository.list_models()

------------------------------------  ------------------------------------------------------------  ------------------------  --------------
GUID                                  NAME                                                          CREATED                   TYPE
05624715-e23d-4a68-a7c7-cdcadb796036  Life Expectancy Prediction - P3 ExtraTreesRegressorEstimator  2020-06-14T05:06:59.935Z  wml-hybrid_0.1
abe81199-5820-4521-b1e2-91ec04a70697  Churn Analysis - P3 LGBMClassifierEstimator                   2020-06-13T16:27:35.341Z  wml-hybrid_0.1
4f80b567-25f4-4c70-a9bd-1e704c7ee834  Churn Analysis - P3 LGBMClassifierEstimator                   2020-06-11T16:33:27.943Z  wml-hybrid_0.1
------------------------------------  ------------------------------------------------------------  ------------------------  --------------


## Accessing the deployed model
We extract the information on deployed models and find the deployment for **`Churn model deployment`**.

In [6]:
# List the deployed models
client.deployments.list()

------------------------------------  --------------------------  -----  ------------------------  -------------
GUID                                  NAME                        STATE  CREATED                   ARTIFACT_TYPE
41df2241-7d7d-4002-bca2-54cfa88157dc  Life Expectancy Deployment  ready  2020-06-14T05:07:40.313Z  model
b825aca9-c709-4348-8bf2-777fd1a9f19f  Churn Analysis Deployment   ready  2020-06-11T16:37:27.845Z  model
------------------------------------  --------------------------  -----  ------------------------  -------------


In [7]:
# Extract the information for "Churn model deployment" 
deployments_details = client.deployments.get_details()
deployed_uid = next(item for item in deployments_details['resources'] 
                    if item['entity']["name"] == "Life Expectancy Deployment")['metadata']['guid']

### Score a record

In [ ]:
# Execute the model
scoring_payload = { client.deployments.ScoringMetaNames.INPUT_DATA: 
                   [{
                     'fields': ['Country','Year','Status','Adult Mortality','infant deaths','Alcohol',
                                'percentage expenditure','Hepatitis B','Measles','BMI','under-five deaths',
                                'Polio','Total Expernd','LongDistanceBilltype',
                                'Usage','RatePlan'], 
                     'values': [[1,'F','S',1.0,38000.0,'N',24.393333,23.56,0.0,206.08,0.0,'CC','Budget','Intnl_discount',229.64,3.0],                      
                                [6,'M','M',2.0,29616.0,'N',49.426667,29.78,0.0,45.5,0.0,  'CH','FreeLocal','Standard',75.29,2.0]
                               ]
                    }]
                  }
predictions = client.deployments.score(deployed_uid, scoring_payload)


In [ ]:
predictions

In [ ]:
for prediction in predictions['predictions'] :
    for result in prediction['values'] :
        print('Prediction: ' + str(result[0]) + ", probability: [" + 
              str(result[1][0]) + ', ' +  str(result[1][1]) + "]" )


## Removing a deployed model
We can remove a model from the repository using the remove method.
In the example below, we remove the deployed model we just used since we already have the uid.
We could look over like we did before and remove all the deployed models.

A similar delete operation can be used to remove the saved models.

In [8]:
# Display the list first
client.deployments.list()

------------------------------------  --------------------------  -----  ------------------------  -------------
GUID                                  NAME                        STATE  CREATED                   ARTIFACT_TYPE
41df2241-7d7d-4002-bca2-54cfa88157dc  Life Expectancy Deployment  ready  2020-06-14T05:07:40.313Z  model
b825aca9-c709-4348-8bf2-777fd1a9f19f  Churn Analysis Deployment   ready  2020-06-11T16:37:27.845Z  model
------------------------------------  --------------------------  -----  ------------------------  -------------


In [ ]:
# Remove the deployment
client.deployments.delete(deployed_uid)

In [ ]:
client.deployments.list()